If you don't already have `pysyd`, see steps to [install](https://pysyd.readthedocs.io/en/latest/installation.html):
  1. `python -m pip install pysyd`
  2. `mkdir pysyd`
  3. `cd pysyd`
  4. `pysyd setup`
  
The last step will set up your local `pysyd` folder with the appropriate directory structure that works best with the software *and* download some examples. Let's run a couple of them!

In [ ]:
import pysyd

## TL;DR
`pySYD` is adapted from the IDL-based asteroseismic pipeline, `SYD`, which has been a very powerful for automating the detection of solar-like oscillations. The analysis is basically two-fold:

 1. remove crude background fit and estimate numax
 2. remove proper background fit and estimate numax (+ other things)

### Step 1: load default parameters

This step is analogous to running software via command-line, which has a lot of these parameters set to their defaults. Most of them should never need to be touched... unless you have a giant star with low numax.

In [ ]:
from pysyd.utils import Parameters

# KIC 1435467
name, args = '1435467', Parameters()

print(args.__dict__.keys())

### Step 2: add target(s)

This will draw from information (via stars_info.csv) when available and then pull the rest of the needed parameters from the defaults to create a new (sub)dictionary for each target.

In [ ]:
args.add_targets(stars=[name])

print(args.params[name])

### Step 3: create pipeline target

Now that we have our two main ingredients we can create a `Target` object, which is where a bulk of the analysis will occur. This will essentially pop that (sub)dictionary from above and then attempt to load in the data.

In [ ]:
from pysyd.target import Target

star = Target(name, args)
# Let's change some defaults
star.params['show'] = True
star.params['verbose'] = True
cont = star.load_data()

If it has done so successfully, the verbose output (i.e. above) will tell you about the files it loaded in and automatically calculate if the power spectrum is over- or critically-sampled. From this point, the code works in those two steps I mentioned at the beginning.

### Step 4: crude background fit

In [ ]:
# First go
star.estimate_parameters()

-> insert first thing that can go wrong

Here we know the numax for Estimate (3) is basically correct since it's fairly high S/N but it's selecting the first model. We can get around this through the `ask` parameter, which literally means to ask you which model to select instead. More generally, this is a helpful parameter to use if you think something is going wrong in this first step because it will output a figure that can help diagnose most issues.

In [ ]:
# Second go
name, args = '1435467', Parameters()
args.add_targets(stars=[name])
star = Target(name, args)
star.params['show'] = True
star.params['verbose'] = True
star.params['ask'] = True
cont = star.load_data()
star.estimate_parameters()

And then we have our final scenario... the real scenario.

Let's say that I know I've used a high-pass filter so the low frequencies cannot be trusted and that there is also a high-frequency artefact that I want the program to ignore. 

In [ ]:
# Third go
name, args = '1435467', Parameters()
args.add_targets(stars=[name])
star = Target(name, args)
star.params['show'] = True
star.params['verbose'] = True
star.params['lower_ex'] = 100.0
star.params['upper_ex'] = 6000.0
cont = star.load_data()
star.estimate_parameters()

So now that we are absolutely sure we have a good estimate for numax, we can let the software do its thing... and derive the stellar background and global asteroseismic parameters.

FWIW the reason this is done in a two-step process is because for solar-like oscillations, we know there is localized power due to the oscillations but there is also various red and white noise components which can affect the reliability of yours results. Therefore for the proper background fit in this next step, the region of oscillations is actually masked out for that very reason.

### Step 5: derive all parameters

In [ ]:
star.derive_parameters()

In [ ]:
star.show_results()

### Putting this all together

In [ ]:
name, args = '1435467', Parameters()
args.add_targets(stars=[name])
star = Target(name, args)
star.params['show'] = True
star.params['lower_ex'] = 100.0
star.params['upper_ex'] = 6000.0
cont = star.load_data()
star.estimate_parameters()
star.derive_parameters()
star.show_results()

## A more evolved star

In [ ]:
name, args = '11618103', Parameters()
args.add_targets(stars=[name])
star = Target(name, args)
star.params['show'] = True
star.params['verbose'] = True
cont = star.load_data()
star.estimate_parameters()
star.derive_parameters()
star.show_results()

In [ ]:
name, args = '11618103', Parameters()
args.add_targets(stars=[name])
star = Target(name, args)
star.params['show'] = True
star.params['verbose'] = True
star.params['noy'] = '7+1'
cont = star.load_data()
star.estimate_parameters()
star.derive_parameters()
star.show_results()

## Even more evolved?

Things that are affected at lower frequencies:
1. crude background fit 
   - smoothing (`--sw`, `--smoothwidth`) of power spectrum (green/red lines, panel 2 of 6 in first figure)
   - adjust (parameter) option for more evolved stars (or some similar keyword that allows you to provide 'MS', 'SG', 'RG' or something similar)
   
2. main parameter derivation
   - box filter (`--bf`, `--box`) only affects plotting, does not affect results
   - independent width (`--iw`, `--indwidth`) acts the same as the smoothing width does in the first module and is also in muHz
   - smoothed power spectrum (`--sp`, `--smoothps`) is a box filter that smooths out the power spectrum before computing the ACF and *does* affect your analysis (middle, middle panel 5 of 9 panel figure)

In [ ]:
name, args = '11618103', Parameters()
args.add_targets(stars=[name])
star = Target(name, args)
star.params['show'] = True
star.params['verbose'] = True
# first module
star.params['smooth_width'] = 1.0
star.params['lower_ex'] = 1.0
# second module
star.params['ind_width'] = 1.0
star.params['smooth_ps'] = 0.1
star.params['lower_bg'] = 1.0
cont = star.load_data()
star.estimate_parameters()
star.derive_parameters()
star.show_results()